In [61]:
import pandas as pd
import sys
import os

In [62]:
DEBUG = False

# Função para adicionar o caminho do módulo 'utils' ao sys.path
utils_path = os.path.abspath("../") if DEBUG else os.path.abspath("../../")
if utils_path not in sys.path:
    sys.path.append(utils_path)

In [63]:
from utils.auxiliary_functions.all_auxiliary_functions import (
    extrair_dados_database,
    carregar_dados_database,
)

In [64]:
def calcular_periodo_integralizado(periodo_atual, quant_integralizado):
    ano, semestre = map(int, periodo_atual.split('/'))
    quant_integralizado = quant_integralizado
    # Se quant_integralizado for ímpar, ajusta semestre e ano
    if quant_integralizado % 2 == 1:
        semestre += 1
        if semestre > 2:
            semestre = 1
            ano += 1
    
    # Calcula o novo ano considerando todos os semestres integralizados
    ano += quant_integralizado // 2

    return f"{ano}/{semestre}"

def obter_ultimo_periodo_cursado(row):
    disciplinas_str = row.get("disciplinasCursadas", "")
    periodos_trancados = row.get("periodosTrancados", "")
    situacao_matricula = row.get("situacaoMatricula", "")
    cra_periodo = row.get("craPorPeriodo", "")
    cr_periodo = row.get("crPorPeriodo", "")
    periodo_ingresso_ufrj = row.get("periodoIngressoUFRJ", "")

    # Caso disciplinas cursadas seja NaN ou vazio
    if pd.isna(disciplinas_str) or disciplinas_str == "":
        if pd.isna(cra_periodo) and pd.isna(cr_periodo):
            return periodo_ingresso_ufrj
        else:
            return cra_periodo.split(' - ')[0]

    # Extração dos períodos cursados
    disciplinas = disciplinas_str.split("\n")
    periodos = [disciplina.split(" - ")[0] for disciplina in disciplinas]

    # Encontrar o maior período cursado
    maior_periodo = max(periodos)

    # Considerar períodos trancados se a situação da matrícula for "Cancelada"
    if situacao_matricula == "Cancelada" and not pd.isna(periodos_trancados):
        trancados = periodos_trancados.replace(" ", "").split("|")
        maior_periodo_trancado = max(trancados)
        # Calcula o periodo seguinte ao trancado
        maior_periodo_trancado = calcular_periodo_integralizado(maior_periodo_trancado, 1)
        maior_periodo = max(maior_periodo, maior_periodo_trancado)

    return maior_periodo.replace("/0", "/1")

# Processar Trancamentos
def processar_trancamentos(row):
    if pd.isna(row):
        return row
    row = row.replace(' ', '')
    row = tuple(row.split('|'))
    return row

In [65]:
# Leitura do arquivo
df_silver = extrair_dados_database("alunos_ufrj.parquet", "silver")

# Carregando as dimensoes
df_d_aluno = extrair_dados_database("d_aluno.parquet", "gold")
df_d_periodo = extrair_dados_database("d_periodo.parquet", "gold")
df_d_curso = extrair_dados_database("d_curso.parquet", "gold")
df_d_situacao = extrair_dados_database("d_situacao.parquet", "gold")

In [66]:
mapeamento_situacao_matricula = {
    'Ativa': 'Ativa',
    'Rematrícula por destrancamento automático' : 'Ativa',
    'Rematrícula por ativação do segmento referente via AGF' : 'Ativa',
    'Rematrícula por destrancamento ou descancelamento' : 'Ativa',
    'Cancelada por abandono': 'Cancelada',
    'Cancelada por abandono Definitivo': 'Cancelada',
    'Cancelada por morte': 'Cancelada',
    'Cancelada por outros motivos': 'Cancelada',
    'Cancelada por rendimento escolar insuficiente': 'Cancelada',
    'Cancelada por transferência': 'Cancelada',
    'Cancelada por ultrapassar prazo de integralização': 'Cancelada',
    'Cancelamento a pedido': 'Cancelada',
    'Cancelamento por conclusão de Mobilidade Acadêmica': 'Cancelada',
    'Cancelamento por decisão judicial': 'Cancelada',
    'Cancelamento por opção de curso': 'Cancelada',
    'Cancelamento por opção de instituição': 'Cancelada',
    'Exclusão Lógica': 'Cancelada',
    'Cancelamento por ultrapassagem do prazo máximo de trancamento': 'Cancelada',
    'Trancada': 'Trancada',
    'Trancamento Solicitado': 'Trancada',
    'Trancamento automático: Perigo de cancelamento imediato': 'Trancada',
    'Cancelada por conclusão de curso': 'Concluido'
}

df_silver['situacaoMatricula'] = df_silver['situacaoMatriculaAtual'].map(mapeamento_situacao_matricula)

In [67]:
# Obtém o valor máximo da coluna 'periodoIngressoUFRJ'
periodo_atual = df_silver['periodoIngressoUFRJ'].max()

# Filtra os dados para obter apenas as linhas com 'situacao_matricula' igual a 'Ativa'
df_situacao_atual = df_silver[(df_silver['situacaoMatricula'] == "Ativa") | (df_silver['situacaoMatricula'] == "Trancada")].copy()
df_cancel_concluido = df_silver[(df_silver['situacaoMatricula'] == "Cancelada") | (df_silver['situacaoMatricula'] == "Concluido")].copy()

# Preparando dataframe de Trancamento
df_trancada = df_silver[~df_silver['periodosTrancados'].isna()].copy()
df_trancada['situacaoMatricula'] = "Trancada"
df_trancada['situacaoMatriculaAtual'] = df_trancada['situacaoMatriculaAtual'].apply(lambda situacao: situacao if situacao.lower().startswith("trancamento") else 'Trancada')
df_trancada['periodo'] = df_trancada['periodosTrancados'].apply(processar_trancamentos)

In [68]:
# Definido os 3 tipos de dataframes
df_situacao_atual['periodo'] = periodo_atual
df_trancada = df_trancada.explode('periodo')
df_cancel_concluido['periodo'] = df_cancel_concluido.apply(obter_ultimo_periodo_cursado, axis=1)

In [69]:
colunas_tb_fato = [
    "matriculaDre",
    "periodo",
    "codCursoIngresso",
    "cursoIngressoUFRJ",
    "codCursoAtual",
    "cursoAtual",
    "situacaoMatriculaAtual",
]

df_ativa = df_situacao_atual[colunas_tb_fato]
df_trancada = df_trancada[colunas_tb_fato]
df_cancel_concluido = df_cancel_concluido[colunas_tb_fato]

# Usa a função concat para unir os DataFrames
df_final = pd.concat([df_ativa, df_trancada, df_cancel_concluido], axis=0)

# Reseta o índice se necessário
df_final.reset_index(drop=True, inplace=True)

In [70]:
# Padronizando nome das colunas
df_final = df_final.rename(
    columns={
        "matriculaDre": "DS_MATRICULA_DRE",
        "periodo": "DS_PERIODO",
        "codCursoIngresso": "CD_CURSO_INGRESSO",
        "cursoIngressoUFRJ": "DS_NOME_CURSO_INGRESSO",
        "codCursoAtual": "CD_CURSO_ATUAL",
        "cursoAtual": "DS_NOME_CURSO_ATUAL",
        "situacaoMatriculaAtual": "DS_SITUACAO_DETALHADA",
    }
)

In [71]:
# Mescla com a tabela D_PERIODO, selecionando apenas a coluna 'sk_d_periodo'
f_situacao_metricula = pd.merge(
    df_final, df_d_periodo[["DS_PERIODO", "SK_D_PERIODO"]], on="DS_PERIODO", how="inner"
)

# Mescla com a tabela D_CURSO
f_situacao_metricula = pd.merge(
    f_situacao_metricula,
    df_d_curso[["SK_D_CURSO", "CD_CURSO_INGRESSO", "CD_CURSO_ATUAL"]],
    on=["CD_CURSO_INGRESSO", "CD_CURSO_ATUAL"],
    how="inner",
)

# Mescla com a tabela D_ALUNO e remove as colunas desnecessárias
f_situacao_metricula = pd.merge(
    f_situacao_metricula,
    df_d_aluno[['SK_D_ALUNO', 'DS_MATRICULA_DRE']],
    on="DS_MATRICULA_DRE",
    how="inner",
)

# Mescla com a tabela D_SITUACAO e remove as colunas desnecessárias
f_situacao_metricula = pd.merge(
    f_situacao_metricula,
    df_d_situacao[['SK_D_SITUACAO', 'DS_SITUACAO_DETALHADA']],
    on="DS_SITUACAO_DETALHADA",
    how="inner",
)

In [72]:
f_situacao_metricula = f_situacao_metricula[['SK_D_ALUNO', 'SK_D_PERIODO', 'SK_D_CURSO', 'SK_D_SITUACAO']]

In [73]:
# Salvar dados na camada Gold
carregar_dados_database(f_situacao_metricula, "f_situacao_metricula.parquet", "gold")